<a href="https://colab.research.google.com/github/crunchdomo/llm_conversation/blob/main/test_base_openai_baselne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import openai
import json
from datetime import datetime

# Initialize OpenAI client
client = openai.OpenAI(api_key="")  # Replace with your actual API key

def cook_recipe(recipe_data, automated_inputs):
    # Initialize conversation
    chat = [
        {
            "role": "system",
            "content": """You are a master chef. Guide the user through the recipe step-by-step. For each step:
                        - Use metric units.
                        - Number each step.
                        - End each step with 'Awaiting confirmation.'
                        - Do not repeat previous steps or instructions.
                        - After the last step, say 'COMPLETE'.
                      """
        }
    ]

    # Structured recipe submission
    chat.append({
        "role": "user",
        "content": f"""Guide me through this recipe step-by-step:

        TITLE: {recipe_data['title']}

        INGREDIENTS:
        {chr(10).join(recipe_data['ingredients'])}

        INSTRUCTIONS:
        {recipe_data['instructions']}"""
    })

    # Process automated inputs
    for user_input in automated_inputs:
        # Generate response
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=chat,
            max_tokens=512,
            temperature=0.7
        )
        chef_response = response.choices[0].message.content
        print(f"\nChef: {chef_response}")
        chat.append({"role": "assistant", "content": chef_response})

        # Check for 'COMPLETE' in chef's response
        if "COMPLETE" in chef_response.lower():
            break

        # Add automated input
        print(f"\nTrainee: {user_input}")
        chat.append({"role": "user", "content": user_input})

        if "exit" in user_input.lower():
            break


    save_conversation(chat)
    return chat

def save_conversation(chat_history, filename=None):
    if not filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"cooking_session_{timestamp}.json"

    with open(filename, 'w') as f:
        json.dump(chat_history, f, indent=2)
    print(f"Saved to {filename}")

# Recipe data structure
recipe = {
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions": """
        1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

        2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.

        3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

        4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
        Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

        5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

        6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

        7. Using your fingers, mash flour and butter in a small bowl to combine.

        8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

        9. Serve chicken with gravy and squash panzanella alongside.
            """
}

# Predefined automated trainee inputs
automated_inputs = [
    "Got it! Ready for step 1.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "Done Continue.",
    "exit"
]

# Start cooking session with automated inputs
cook_recipe(recipe, automated_inputs)



Chef: 1. Begin by patting the chicken dry with paper towels. Season it all over with 2 teaspoons of kosher salt and tie the legs together with kitchen twine. Allow the chicken to sit at room temperature for 1 hour. Awaiting confirmation.

Trainee: Got it! Ready for step 1.

Chef: 2. While the chicken sits, proceed with the squash. Halve the acorn squash and scoop out the seeds. Using a vegetable peeler, remove the skin along the ridges of each squash half. Then, cut each half into ½-inch thick wedges and arrange them on a rimmed baking sheet. Awaiting confirmation.

Trainee: Done Continue.

Chef: 3. In a large bowl, combine the chopped sage, chopped rosemary, and 6 tablespoons of melted butter. Pour half of this herb butter mixture over the squash on the baking sheet. Sprinkle the squash with the allspice, a pinch of crushed red pepper flakes, and ½ teaspoon of salt. Season additionally with freshly ground black pepper to taste. Toss everything to ensure the squash is evenly coated. A

[{'role': 'system',
  'content': "You are a master chef. Guide the user through the recipe step-by-step. For each step:\n                        - Use metric units.\n                        - Number each step.\n                        - End each step with 'Awaiting confirmation.'\n                        - Do not repeat previous steps or instructions.\n                        - After the last step, say 'COMPLETE'.\n                      "},
 {'role': 'user',
  'content': 'Guide me through this recipe step-by-step:\n\n        TITLE: Miso-Butter Roast Chicken With Acorn Squash Panzanella\n\n        INGREDIENTS:\n        1 (3½–4-lb.) whole chicken\n2¾ tsp. kosher salt, divided, plus more\n2 small acorn squash (about 3 lb. total)\n2 Tbsp. finely chopped sage\n1 Tbsp. finely chopped rosemary\n6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\n¼ tsp. ground allspice\nPinch of crushed red pepper flakes\nFreshly ground black pepper\n⅓ loaf good-quality sturdy white bread, torn int